In [3]:
import pandas as pd
import numpy as np

In [12]:
#ref

# dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

# df = pd.read_csv(infile, parse_dates=['datetime'], date_parser=dateparse)

In [13]:
dateparse = lambda x: pd.datetime.strptime(x, '%m-%d-%Y %H:%M:%S')
    

trump_df = pd.read_csv(r"D:\Documents\Docs_oldnew\trump_tweet_api\Trump_tweets_China_since2016.txt", parse_dates=['created_at'], date_parser=dateparse)
trump_df.head()

,text,created_at
0,.@FoxNews “Trump wins major victories in Chin...,2019-12-17 14:48:50
1,....energy independence manufacturing resilien...,2019-12-16 05:28:18
2,....energy independence manufacturing resilien...,2019-12-16 05:05:08
3,Chuck Schumer sat for years during the Obama A...,2019-12-14 19:09:26
4,We have agreed to a very large Phase One Deal ...,2019-12-13 15:25:47


In [14]:
trump_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
text          365 non-null object
created_at    365 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 5.8+ KB


In [15]:
len(trump_df)

365

In [16]:
nonrepeat_text =trump_df[['text']].drop_duplicates()

In [17]:
nonrepeat_text.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            355, 356, 357, 358, 359, 360, 361, 362, 363, 364],
           dtype='int64', length=358)

In [18]:
nonrepeat_text.head()

,text
0,.@FoxNews “Trump wins major victories in Chin...
1,....energy independence manufacturing resilien...
2,....energy independence manufacturing resilien...
3,Chuck Schumer sat for years during the Obama A...
4,We have agreed to a very large Phase One Deal ...


In [19]:
nonrepeat_datetime = trump_df['created_at'][nonrepeat_text.index]

#account for timezone difference btw NY and HK
time_zoneadjusted =nonrepeat_datetime + pd.DateOffset(hours= +13)

In [20]:
#object type of datetime column
type(nonrepeat_datetime[0])


pandas._libs.tslibs.timestamps.Timestamp

In [21]:
nonrepeat_text['record_time'] = time_zoneadjusted
nonrepeat_text.head()
#neccessary to make it into index?? so as to do nested index filtering?

,text,record_time
0,.@FoxNews “Trump wins major victories in Chin...,2019-12-18 03:48:50
1,....energy independence manufacturing resilien...,2019-12-16 18:28:18
2,....energy independence manufacturing resilien...,2019-12-16 18:05:08
3,Chuck Schumer sat for years during the Obama A...,2019-12-15 08:09:26
4,We have agreed to a very large Phase One Deal ...,2019-12-14 04:25:47


In [22]:
#try extracting the hour
#nonrepeat_text['record_time'].dt.hour

#? use df.between_time('00:00', '6:00') ## this is filtering on a certain criteria, not slicing?? 

In [23]:
## to-do:
# split into a column in date, another in time

In [24]:
#sort the datetime index in natural order

adjusted_df = nonrepeat_text.set_index('record_time').sort_index()
adjusted_df.head()

,text
record_time,
2016-05-09 09:15:20,Crooked Hillary just can't close the deal with...
2016-05-14 14:03:02,If Crooked Hillary Clinton can't close the dea...
2016-05-18 00:37:03,The pathetic new hit ad against me misrepresen...
2016-05-18 00:46:22,Crooked Hillary Clinton put out an ad where I ...
2016-05-20 22:47:09,Crooked Hillary has zero imagination and even ...


In [26]:
adjusted_df =adjusted_df.reset_index()

In [27]:
# get the date component from datetime object
adjusted_df['record_time'][0].date()
#get the time component 
adjusted_df['record_time'][0].time()

datetime.time(9, 15, 20)

In [28]:
#adjusted_df['year_ind'] =adjusted_df.index.year
#adjusted_df['mon_ind'] =adjusted_df.index.month
#adjusted_df['day_ind'] =adjusted_df.index.day
#adjusted_df['day_ind'].head()
#adjusted_df.to_datetime([adjusted_df['year_ind'], adjusted_df['mon_ind'], adjusted_df['day_ind']])

#pd.to_datetime(adjusted_df.index, format='%Y%m%d', errors='ignore')

#t.apply(lambda x: x.date()) 

adjusted_df['date']=adjusted_df['record_time'].apply(lambda x: x.date())
adjusted_df['time']=adjusted_df['record_time'].apply(lambda x: x.time())
adjusted_df['day_of_week'] = adjusted_df['record_time'].apply(lambda x:x.day_name())

In [29]:
adjusted_df.head()

,record_time,text,date,time,day_of_week
0,2016-05-09 09:15:20,Crooked Hillary just can't close the deal with...,2016-05-09,09:15:20,Monday
1,2016-05-14 14:03:02,If Crooked Hillary Clinton can't close the dea...,2016-05-14,14:03:02,Saturday
2,2016-05-18 00:37:03,The pathetic new hit ad against me misrepresen...,2016-05-18,00:37:03,Wednesday
3,2016-05-18 00:46:22,Crooked Hillary Clinton put out an ad where I ...,2016-05-18,00:46:22,Wednesday
4,2016-05-20 22:47:09,Crooked Hillary has zero imagination and even ...,2016-05-20,22:47:09,Friday


In [30]:
adjusted_df = adjusted_df.drop(columns = ['record_time']).set_index('date')

In [31]:
adjusted_df = adjusted_df[['day_of_week','time','text']]
adjusted_df.head()

,day_of_week,time,text
date,,,
2016-05-09,Monday,09:15:20,Crooked Hillary just can't close the deal with...
2016-05-14,Saturday,14:03:02,If Crooked Hillary Clinton can't close the dea...
2016-05-18,Wednesday,00:37:03,The pathetic new hit ad against me misrepresen...
2016-05-18,Wednesday,00:46:22,Crooked Hillary Clinton put out an ad where I ...
2016-05-20,Friday,22:47:09,Crooked Hillary has zero imagination and even ...


In [32]:
HSCE_df =pd.read_csv(r'D:\Documents\Docs_oldnew\trump_tweet_api\HSCE_since2016May.csv', index_col = 0, parse_dates = True)
HSCE_df.head()

,open,high,low,close,volume
date,,,,,
2016-05-09,20279.90039,20303.18945,20125.77930,20156.81055,1323285400
2016-05-10,19962.42969,20294.46094,19962.42969,20242.67969,1446925800
2016-05-11,20347.75977,20347.75977,20008.91992,20055.28906,1337926500
2016-05-12,20100.50000,20100.50000,19863.61914,19915.46094,1150409200
2016-05-13,19843.49023,19881.91016,19594.60938,19719.28906,1758498600


In [33]:
#HSCE_df['close'].shift(periods =1).head()

In [34]:
HSCE_df['abs_change'] = HSCE_df['close'] - HSCE_df['close'].shift(periods =1)
HSCE_df['pct_change'] = (HSCE_df['abs_change'] / HSCE_df['close'].shift(periods=1))*100 

HSCE_df.head()

,open,high,low,close,volume,abs_change,pct_change
date,,,,,,,
2016-05-09,20279.90039,20303.18945,20125.77930,20156.81055,1323285400,NaN,NaN
2016-05-10,19962.42969,20294.46094,19962.42969,20242.67969,1446925800,85.86914,0.426006
2016-05-11,20347.75977,20347.75977,20008.91992,20055.28906,1337926500,-187.39063,-0.925720
2016-05-12,20100.50000,20100.50000,19863.61914,19915.46094,1150409200,-139.82812,-0.697213
2016-05-13,19843.49023,19881.91016,19594.60938,19719.28906,1758498600,-196.17188,-0.985023


In [35]:
HSCE_df['sign_change'] = HSCE_df['pct_change'] > 0

In [36]:
HSCE_df.head()

,open,high,low,close,volume,abs_change,pct_change,sign_change
date,,,,,,,,
2016-05-09,20279.90039,20303.18945,20125.77930,20156.81055,1323285400,NaN,NaN,False
2016-05-10,19962.42969,20294.46094,19962.42969,20242.67969,1446925800,85.86914,0.426006,True
2016-05-11,20347.75977,20347.75977,20008.91992,20055.28906,1337926500,-187.39063,-0.925720,False
2016-05-12,20100.50000,20100.50000,19863.61914,19915.46094,1150409200,-139.82812,-0.697213,False
2016-05-13,19843.49023,19881.91016,19594.60938,19719.28906,1758498600,-196.17188,-0.985023,False


In [37]:
HSCE_extract =HSCE_df[['close','pct_change','sign_change']]

In [38]:
type(HSCE_extract.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [39]:
#next_day performance is missing if drop_na().. no good to miss those reactions
##to-do: need to check the weekday as well; and holidays

## if his tweet is on Sat and Sun, fill_na with next-day performance

tweet_HSCE_react = pd.merge(adjusted_df,HSCE_extract, how = 'outer', left_index=True, right_index=True)
tweet_HSCE_react.head()

,day_of_week,time,text,close,pct_change,sign_change
date,,,,,,
2016-05-09,Monday,09:15:20,Crooked Hillary just can't close the deal with...,20156.81055,NaN,False
2016-05-10,NaN,NaN,NaN,20242.67969,0.426006,True
2016-05-11,NaN,NaN,NaN,20055.28906,-0.925720,False
2016-05-12,NaN,NaN,NaN,19915.46094,-0.697213,False
2016-05-13,NaN,NaN,NaN,19719.28906,-0.985023,False


In [40]:
#check only tweets after Trump inaugurated

tweets_inau_df =tweet_HSCE_react['2017-01-20':]
tweets_inau_df.head()

,day_of_week,time,text,close,pct_change,sign_change
date,,,,,,
2017-01-20,NaN,NaN,NaN,22885.91016,-0.711718,False
2017-01-23,NaN,NaN,NaN,22898.51953,0.055097,True
2017-01-24,NaN,NaN,NaN,22949.85938,0.224206,True
2017-01-25,NaN,NaN,NaN,23049.11914,0.432507,True
2017-01-26,NaN,NaN,NaN,23374.16992,1.410253,True


In [41]:
tweets_inau_df[tweets_inau_df['day_of_week'] == 'Sunday'].head()

,day_of_week,time,text,close,pct_change,sign_change
date,,,,,,
2017-04-09,Sunday,03:50:59,It was a great honor to have President Xi Jinp...,NaN,NaN,NaN
2017-07-09,Sunday,07:55:23,Leaving Hamburg for Washington D.C. and the WH...,NaN,NaN,NaN
2017-07-30,Sunday,12:29:05,I am very disappointed in China. Our foolish p...,NaN,NaN,NaN
2017-07-30,Sunday,12:35:13,...they do NOTHING for us with North Korea jus...,NaN,NaN,NaN
2017-08-06,Sunday,11:44:34,The United Nations Security Council just voted...,NaN,NaN,NaN


In [42]:
#tweets_inau_df[tweets_inau_df['day_of_week'] == 'Sunday'][['close']].fillna(value= close_values)

In [43]:
##this need a shift of two? or a shift of business_day

from pandas.tseries.offsets import BDay
# df.index = df.index.map(lambda x : x + 0*BDay())


#backup: tweets_inau_df[tweets_inau_df['day_of_week'] == 'Saturday'].index.
Sat_shift_newdate= tweets_inau_df[tweets_inau_df['day_of_week'] == 'Saturday'].index.map(lambda x:x+0*BDay())
Sat_shift_newdate

DatetimeIndex(['2017-02-13', '2017-03-20', '2017-04-24', '2017-05-01',
               '2017-05-15', '2017-11-06', '2018-04-09', '2018-04-30',
               '2018-05-07', '2018-07-23', '2018-08-27', '2018-08-27',
               '2018-11-12', '2018-12-10', '2018-12-17', '2018-12-17',
               '2018-12-24', '2019-03-04', '2019-05-06', '2019-05-13',
               '2019-05-13', '2019-05-13', '2019-05-13', '2019-05-13',
               '2019-05-13', '2019-05-13', '2019-05-13', '2019-05-13',
               '2019-05-13', '2019-05-13', '2019-06-10', '2019-07-01',
               '2019-07-15', '2019-07-29', '2019-07-29', '2019-07-29',
               '2019-08-26', '2019-08-26', '2019-08-26', '2019-08-26',
               '2019-08-26', '2019-08-26', '2019-08-26', '2019-08-26',
               '2019-08-26', '2019-08-26', '2019-09-02', '2019-09-09',
               '2019-09-09', '2019-10-07', '2019-10-14', '2019-10-14',
               '2019-10-14', '2019-10-14', '2019-12-09', '2019-12-16',
      

In [44]:
#tweets_inau_df[tweets_inau_df['day_of_week'] == 'Saturday']['close'].fillna(value = close_values)

In [45]:
#len(tweets_inau_df[tweets_inau_df['day_of_week'] == 'Saturday']['close'])

In [46]:
#HSCE_extract.loc['2017-02-13']['close']

#close_values = HSCE_extract.loc[Sat_shift_newdate]['close']

In [47]:
#211 rows × 6 columns

tweets_inau_nona = tweets_inau_df.dropna()
tweets_inau_nona.head()

,day_of_week,time,text,close,pct_change,sign_change
date,,,,,,
2017-03-31,Friday,11:16:17,The meeting next week with China will be a ver...,24111.58984,-0.779800,False
2017-04-12,Wednesday,00:59:04,I explained to the President of China that a t...,24313.50000,0.934219,True
2017-04-12,Wednesday,01:03:43,North Korea is looking for trouble. If China d...,24313.50000,0.934219,True
2017-04-13,Thursday,01:22:22,Had a very good call last night with the Presi...,24261.66016,-0.213214,False
2017-06-21,Wednesday,07:38:11,While I greatly appreciate the efforts of Pres...,25694.58008,-0.574464,False


In [48]:
#to-do: check if time is before 9:30 or happening during market open hours or after market hours

In [49]:
X= tweets_inau_nona['text']

In [50]:
X.head()

date
2017-03-31    The meeting next week with China will be a ver...
2017-04-12    I explained to the President of China that a t...
2017-04-12    North Korea is looking for trouble. If China d...
2017-04-13    Had a very good call last night with the Presi...
2017-06-21    While I greatly appreciate the efforts of Pres...
Name: text, dtype: object

In [51]:
from sklearn.preprocessing import LabelEncoder

In [52]:
le = LabelEncoder() 
tweets_inau_nona.loc[:,'sign_change'] = le.fit_transform(tweets_inau_nona['sign_change'])

C:\Users\gen80\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [53]:
tweets_inau_nona['sign_change'].head()

date
2017-03-31    0
2017-04-12    1
2017-04-12    1
2017-04-13    0
2017-06-21    0
Name: sign_change, dtype: int32

In [54]:
y = tweets_inau_nona['sign_change']

In [55]:
y.head()

date
2017-03-31    0
2017-04-12    1
2017-04-12    1
2017-04-13    0
2017-06-21    0
Name: sign_change, dtype: int32

In [56]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression, LinearRegression

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [60]:
tfidf= TfidfVectorizer()

X_train_tsf = tfidf.fit_transform(X_train).tocsc()
X_test_tsf = tfidf.transform(X_test).tocsc()

In [72]:
tfidf.vocabulary_

{'lavar': 694,
 'you': 1387,
 'could': 274,
 'have': 557,
 'spent': 1143,
 'the': 1226,
 'next': 829,
 'to': 1252,
 '10': 0,
 'years': 1384,
 'during': 378,
 'thanksgiving': 1224,
 'with': 1363,
 'your': 1388,
 'son': 1133,
 'in': 614,
 'china': 211,
 'but': 178,
 'no': 833,
 'nba': 819,
 'contract': 264,
 'support': 1190,
 'remember': 1023,
 'shoplifting': 1097,
 'is': 648,
 'not': 839,
 'little': 718,
 'thing': 1238,
 'it': 651,
 'really': 1001,
 'big': 155,
 'deal': 300,
 'especially': 408,
 'ungrateful': 1297,
 'fool': 474,
 'looks': 728,
 'like': 713,
 'bernie': 146,
 'sanders': 1067,
 'history': 588,
 'sleepy': 1119,
 'joe': 659,
 'biden': 153,
 'pulling': 976,
 'ahead': 64,
 'and': 84,
 'think': 1240,
 'about': 36,
 'only': 862,
 'here': 577,
 'because': 133,
 'of': 849,
 'man': 747,
 'who': 1357,
 'took': 1259,
 'him': 585,
 'off': 850,
 'trash': 1267,
 'heap': 565,
 'president': 947,
 'wants': 1327,
 'badly': 123,
 'what': 1349,
 'market': 757,
 'wanted': 1325,
 'hear': 566,
 

In [71]:
tfidf.idf_

array([4.74360435, 5.43675153, 5.03128643, ..., 4.33813925, 5.43675153,
       4.74360435])

In [70]:
tfidf.get_feature_names()

['10',
 '100',
 '15',
 '15th',
 '16',
 '1990',
 '1st',
 '20',
 '200',
 '2016',
 '2018',
 '21',
 '25',
 '250',
 '27',
 '2nd',
 '30',
 '325',
 '40',
 '45pm',
 '50',
 '500',
 '517',
 '600',
 '62',
 '7000',
 '70th',
 '77000',
 '800',
 '90',
 '91',
 '97uzwyhiju',
 '9t7iq6f3xe',
 'abc',
 'abcpolitics',
 'able',
 'about',
 'absolutely',
 'acceptable',
 'accepting',
 'act',
 'actions',
 'actively',
 'actually',
 'addictive',
 'additional',
 'additionally',
 'adds',
 'adept',
 'administration',
 'administrations',
 'advanced',
 'advantage',
 'advice',
 'after',
 'again',
 'against',
 'aggressive',
 'ago',
 'agree',
 'agreed',
 'agreement',
 'agricultural',
 'agriculture',
 'ahead',
 'alaska',
 'all',
 'allowing',
 'almost',
 'along',
 'already',
 'also',
 'always',
 'am',
 'amazing',
 'amazon',
 'america',
 'american',
 'americanow',
 'americans',
 'among',
 'amounts',
 'amp',
 'an',
 'and',
 'andypuzder',
 'anniversary',
 'announced',
 'announcements',
 'another',
 'any',
 'anybody',
 'anymore

In [73]:
X_train_tsf.shape

(168, 1392)

In [74]:
X_test_tsf.shape

(43, 1392)

In [75]:
bnb =BernoulliNB()
bnb.fit(X_train_tsf,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [76]:
bnb.score(X_train_tsf,y_train)

0.9940476190476191

In [77]:
bnb.score(X_test_tsf,y_test)

0.5581395348837209

In [78]:
bnb.coef_

array([[-3.29583687, -4.39444915, -3.70130197, ..., -3.70130197,
        -4.39444915, -3.70130197]])

In [79]:
bnb.predict(X_test_tsf[0])

array([1])

In [80]:
#this sentence is actually Neutral on market
##to-do: need to add a 'Neutral' class to original labeling of y

X_test[0]

'Doing great with China and other Trade Deals. The only problem we have is Jay Powell and the Fed. He’s like a golfer who can’t putt has no touch. Big U.S. growth if he does the right thing BIG CUT - but don’t count on him! So far he has called it wrong and only let us down....'

In [81]:
lr = LogisticRegression()
lr.fit(X_train_tsf,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [82]:
lr.score(X_test_tsf,y_test)

0.5581395348837209

In [83]:
lr.coef_

array([[-0.01582716, -0.10677973, -0.01282007, ..., -0.14345679,
        -0.10086638, -0.07275792]])

In [84]:
lr.predict_proba(X_test_tsf[0])

array([[0.48328469, 0.51671531]])

In [87]:
vocab = tfidf.get_feature_names()
weights = lr.coef_.flatten()

inds = np.argsort(lr.coef_.flatten())
negative_words = [vocab[index] for index in inds[:20]]
positive_words = [vocab[index] for index in inds[-20:]]
neg_words_weights = [(weights[index]) for index in inds[:20]]
pos_words_weights = [(weights[index]) for index in inds[-20:]]

df = pd.DataFrame({'Neg feats':negative_words, 'Neg weights':neg_words_weights, 
                   'Pos feats':positive_words, 'Pos weights':pos_words_weights})

In [88]:
##comment-- this is not right. need to remove some stopwords; also have to teach it about vocabulary of pos/neg sentiment
df

,Neg feats,Neg weights,Pos feats,Pos weights
0,had,-0.589833,will,0.332488
1,so,-0.563815,soon,0.335913
2,korea,-0.532956,pouring,0.347157
3,no,-0.413554,too,0.353319
4,usa,-0.407309,massive,0.358309
5,chinese,-0.378273,thing,0.361172
6,due,-0.372611,talking,0.361834
7,meeting,-0.357683,are,0.376974
8,which,-0.357247,great,0.383386
9,joe,-0.356715,day,0.385050


In [89]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(max_features=5000,                       
                      stop_words='english')
X_train_counts = vec.fit_transform(X_train) 
X_test_counts = vec.transform(X_test) 

In [90]:
lr2 = LogisticRegression()
lr2.fit(X_train_counts,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [91]:
##ummm... test score is 65% for 2nd run, versus 53% for 1st run... means the random split did something as sample size too small, not that the model is good
lr2.score(X_test_counts,y_test)

0.6511627906976745

In [92]:
lr2.coef_

array([[ 0.00591649, -0.23720465, -0.11067304, ..., -0.16340604,
        -0.17271729, -0.05891027]])

In [93]:
lr2.predict_proba(X_test_counts[0])

array([[0.01270673, 0.98729327]])

In [94]:
vocab = vec.get_feature_names()
weights = lr2.coef_.flatten()

inds = np.argsort(lr2.coef_.flatten())
negative_words = [vocab[index] for index in inds[:20]]
positive_words = [vocab[index] for index in inds[-20:]]
neg_words_weights = [(weights[index]) for index in inds[:20]]
pos_words_weights = [(weights[index]) for index in inds[-20:]]

df2 = pd.DataFrame({'Neg feats':negative_words, 'Neg weights':neg_words_weights, 
                   'Pos feats':positive_words, 'Pos weights':pos_words_weights})
df2

,Neg feats,Neg weights,Pos feats,Pos weights
0,meeting,-0.623297,right,0.429696
1,usa,-0.621369,talks,0.448022
2,korea,-0.599123,productive,0.456048
3,treated,-0.547867,thing,0.479459
4,chinese,-0.528339,agricultural,0.479686
5,currency,-0.502542,making,0.489952
6,countries,-0.472686,farmers,0.491243
7,larger,-0.465369,soon,0.495359
8,playing,-0.453944,pouring,0.509076
9,manufacturing,-0.448933,win,0.536067


In [123]:
##trying to pass a custom vocab dict to tfidf_vectorizer and see if it can achieve better training results 

#pd.read_csv(r'D:\Documents\Docs_oldnew\trump_tweet_api\keywords_count_dict.txt', header = None, usecols = [0])

manual_vocabulary = {}

file = open(r'D:\Documents\Docs_oldnew\trump_tweet_api\keywords_count_dict.txt')
count = 0
for line in file:
    
    #print(line.split(':'))
    manual_vocabulary[line.split(':')[0]] = count
    count +=1

In [124]:
# try adding some manually-defined keywords with weights(custom vocabulary)
manual_vocabulary

{'trade': 0,
 'deal': 1,
 'currency': 2,
 'sell': 3,
 'products': 4,
 'decide': 5,
 'agree': 6,
 'meet': 7,
 'meeting': 8,
 'surplus': 9,
 'billion': 10,
 'money': 11,
 'economy': 12,
 'economic': 13,
 'talk': 14,
 'spoken': 15,
 'call': 16,
 'discuss': 17,
 'appreciate': 18,
 'grew': 19,
 'growth': 20,
 'financial': 21,
 'hopefully': 22,
 'much': 23,
 'forward': 24,
 'stronger': 25,
 'relationship': 26,
 'progress': 27,
 'disappointed': 28,
 'foolish': 29,
 'deficit': 30,
 'not': 31,
 'yet': 32,
 'nothing': 33,
 'little': 34,
 'tax': 35,
 'tariff': 36,
 'incompetence': 37,
 'advantage': 38,
 'loss': 39,
 'losing': 40}

In [125]:
tfidf2 = TfidfVectorizer(vocabulary = manual_vocabulary)

X_train_tsf2 = tfidf2.fit_transform(X_train).tocsc()
X_test_tsf2 = tfidf2.transform(X_test).tocsc()

# ##error:  Vocabulary contains repeated indices.-- so the key-value pair in manual_dict is word-unique index, not word-weights

In [126]:
# actually no idea what idf means
tfidf2.idf_

array([2.60353819, 2.63339115, 3.56494936, 5.43675153, 4.18398857,
       6.12989871, 5.03128643, 5.03128643, 3.93267414, 6.12989871,
       3.82731362, 3.56494936, 4.18398857, 4.5204608 , 6.12989871,
       6.12989871, 4.74360435, 6.12989871, 5.43675153, 5.43675153,
       5.43675153, 5.03128643, 4.5204608 , 2.9944045 , 4.18398857,
       5.43675153, 3.93267414, 4.5204608 , 5.43675153, 6.12989871,
       5.03128643, 2.41632665, 4.74360435, 4.18398857, 4.05045717,
       5.03128643, 4.18398857, 6.12989871, 4.5204608 , 6.12989871,
       4.5204608 ])

In [138]:
#tfidf2.get_feature_names()  ##a list of keys of input manual_vocab

In [135]:
tfidf2.vocabulary_.keys()

dict_keys(['trade', 'deal', 'currency', 'sell', 'products', 'decide', 'agree', 'meet', 'meeting', 'surplus', 'billion', 'money', 'economy', 'economic', 'talk', 'spoken', 'call', 'discuss', 'appreciate', 'grew', 'growth', 'financial', 'hopefully', 'much', 'forward', 'stronger', 'relationship', 'progress', 'disappointed', 'foolish', 'deficit', 'not', 'yet', 'nothing', 'little', 'tax', 'tariff', 'incompetence', 'advantage', 'loss', 'losing'])

In [130]:
#now try train a lr model with tfidf2 fed with custom vocab
### - .- ||| even worse results, probably because the vocab could not cover all words in tweets
lr_md = LogisticRegression()
lr_md.fit(X_train_tsf2,y_train)
print(lr_md.score(X_train_tsf2,y_train))
print(lr_md.score(X_test_tsf2,y_test))


0.6547619047619048
0.4883720930232558
[[ 0.11357459  0.32641847 -0.86159796  0.3133247   0.74477181  0.
  -0.05639774  0.58785543 -0.65806753  0.         -0.18276183  0.82230415
   0.24888205 -0.43205467  0.          0.          0.24923388  0.
  -0.34444241 -0.3332092   0.37110324 -0.12295741 -0.38391746 -0.01469842
  -0.17101154  0.27180971  0.3768636   1.10994859  0.42266121  0.
  -0.03957254 -0.16256846 -0.6046635  -0.56815593 -0.55920806 -0.62284919
  -0.13686533  0.         -0.39993266  0.          0.42891908]]


In [146]:
#pd.DataFrame({"vocabs":tfidf2.get_feature_names()})

In [145]:
#lr_md.coef_[0]

In [147]:
##ummm these weights are also counter-intuitive for sentiment reactions. probably the stock market is just not correlated or even opposite to sentiment

manualdict_coefs_df =pd.DataFrame({"vocabs":tfidf2.get_feature_names(), 
              "weights":lr_md.coef_[0]})
manualdict_coefs_df

,vocabs,weights
0,trade,0.113575
1,deal,0.326418
2,currency,-0.861598
3,sell,0.313325
4,products,0.744772
5,decide,0.000000
6,agree,-0.056398
7,meet,0.587855
8,meeting,-0.658068
9,surplus,0.000000


In [273]:
from sklearn.linear_model import LinearRegression

In [274]:
## now testing regressor, not classifiers

lr_r = LinearRegression()

In [275]:
lr_r.fit(X_train_tsf,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [277]:
lr_r.score(X_train_tsf,y_train)

0.9999999999999948

In [ ]:
##what does it mean to have a negative score?

In [276]:
lr_r.score(X_test_tsf,y_test)

-0.026509743184107037

In [279]:
lr_r2 = LinearRegression()

lr_r2.fit(X_train_counts,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [280]:
lr_r2.score(X_test_counts,y_test)

-0.08838969175149991

In [282]:
lr_r2.predict(X_test_counts[0])

array([0.26942588])

In [ ]:
## it seems there's not much valid labeling here.... so learning result is bad
# to-do: need to filter more to have better training
#also have to check the correlation of keywords and market response

In [ ]:
##also need to narrow the analysis period to after trade war-- and forever negotiating trade deals-- started. 

In [ ]:
## wikipedia
# 2018
# January 22: Trump announced tariffs on solar panels and washing machines.[80] About 8% of American solar panel imports in 2017 came from China.[81] Imports of residential washing machines from China totaled about $1.1 billion in 2015.[82]
# March 1: Trump announced steel and aluminum tariffs on imports from all countries.[83] The United States had imported about 3% of its steel from China.[84]
# March 22: Trump asked the United States trade representative (USTR) to investigate applying tariffs on US$50–60 billion worth of Chinese goods.[85][86][87] He relied on Section 301 of the Trade Act of 1974 for doing so, stating that the proposed tariffs were "a response to the unfair trade practices of China over the years", including theft of U.S. intellectual property.[88][85] Over 1,300 categories of Chinese imports were listed for tariffs, including aircraft parts, batteries, flat-panel televisions, medical devices, satellites, and various weapons.[89][90]

In [ ]:
## OH suddenly realize I should use a pretrained sentiment model, 
#coz there're so many words with obvious sentiment that may come up in this new dataset(of Trump)
# but obviously his unique use of words also need fit(), as he often uses good words to deliver opposite meaning

In [ ]:
#trying out textblob classifier, training on the 300+ entries dataset related to China
#should expand to use a larger dataset to fully LEARN talk style of Trump on various occasions


In [4]:
trump_clf_df =pd.read_csv('trump_manuallabeling_alldone.csv')

In [5]:
trump_clf_df.head()

,text,created_at,sentiment(0-9),trade-related(y/n)
0,China has been taking out massive amounts of m...,1/2/2017 23:47,1,y
1,The failing @nytimes does major FAKE NEWS Chin...,2/10/2017 13:35,6,n
2,North Korea is behaving very badly. They have ...,3/17/2017 13:07,2,n
3,The meeting next week with China will be a ver...,3/30/2017 22:16,2,y
4,It was a great honor to have President Xi Jinp...,4/8/2017 14:50,7,n


In [7]:
trump_clf_labels =trump_clf_df.drop(columns = ['created_at','trade-related(y/n)'])

In [9]:
trump_clf_labels .head()

,text,sentiment(0-9)
0,China has been taking out massive amounts of m...,1
1,The failing @nytimes does major FAKE NEWS Chin...,6
2,North Korea is behaving very badly. They have ...,2
3,The meeting next week with China will be a ver...,2
4,It was a great honor to have President Xi Jinp...,7


In [11]:
len(trump_clf_labels)

326

In [8]:
#need to do train_test_split?
from sklearn.model_selection import train_test_split

In [10]:
from textblob.classifiers import NaiveBayesClassifier

In [13]:
for row in trump_clf_labels.iterrows():
    print(row)

(0, text              China has been taking out massive amounts of m...
sentiment(0-9)                                                    1
Name: 0, dtype: object)
(1, text              The failing @nytimes does major FAKE NEWS Chin...
sentiment(0-9)                                                    6
Name: 1, dtype: object)
(2, text              North Korea is behaving very badly. They have ...
sentiment(0-9)                                                    2
Name: 2, dtype: object)
(3, text              The meeting next week with China will be a ver...
sentiment(0-9)                                                    2
Name: 3, dtype: object)
(4, text              It was a great honor to have President Xi Jinp...
sentiment(0-9)                                                    7
Name: 4, dtype: object)
(5, text              I explained to the President of China that a t...
sentiment(0-9)                                                    5
Name: 5, dtype: object)
(6, text        

In [30]:
import csv

train_textlist = []

with open('trump_manuallabeling_2cols.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        #print(tuple(row))
        #print(', '.join(row))
        train_textlist.append(tuple(row))
        
train_textlist[1:]

[("China has been taking out massive amounts of money &amp; wealth from the U.S. in totally one-sided trade but won't help with North Korea. Nice!",
  '1'),
 ('"The failing @nytimes does major FAKE NEWS China story saying ""Mr.Xi has not spoken to Mr. Trump since Nov.14."" We spoke at length yesterday!"',
  '6'),
 ('"North Korea is behaving very badly. They have been ""playing"" the United States for years. China has done little to help!"',
  '2'),
 ('The meeting next week with China will be a very difficult one in that we can no longer have massive trade deficits...',
  '2'),
 ('It was a great honor to have President Xi Jinping and Madame Peng Liyuan of China as our guests in the United States. Tremendous...',
  '7'),
 ('I explained to the President of China that a trade deal with the U.S. will be far better for them if they solve the North Korean problem!',
  '5'),
 ('North Korea is looking for trouble. If China decides to help that would be great. If not we will solve the problem wi

In [31]:
len(train_textlist)

327

In [32]:
cl = NaiveBayesClassifier(train_textlist[1:201])

In [35]:
train_textlist[201]

('....This money will come from the massive Tariffs being paid to the United States for allowing China and others to do business with us. The Farmers have been â€œforgottenâ€\x9d for many years. Their time is now!',
 '4')

In [36]:
cl.classify(train_textlist[201][0])

'3'

In [43]:
total_deviation = 0
for tweet in train_textlist[201:]:
    print("sentiment gauge guess: ",cl.classify(tweet[0]))
    print("actual sentiment manual label:", tweet[1])
    deviation = int(cl.classify(tweet[0])) - int(tweet[1])
    print(deviation)
    total_deviation += deviation
    print("total_deviation",total_deviation)  

  
print("avg deviation:",total_deviation/len(train_textlist[201:]))
    
##need  an summary accuracy score here

sentiment gauge guess:  3
actual sentiment manual label: 4
-1
total_deviation -1
sentiment gauge guess:  3
actual sentiment manual label: 3
0
total_deviation -1
sentiment gauge guess:  6
actual sentiment manual label: 2
4
total_deviation 3
sentiment gauge guess:  6
actual sentiment manual label: 5
1
total_deviation 4
sentiment gauge guess:  3
actual sentiment manual label: 2
1
total_deviation 5
sentiment gauge guess:  3
actual sentiment manual label: 4
-1
total_deviation 4
sentiment gauge guess:  3
actual sentiment manual label: 3
0
total_deviation 4
sentiment gauge guess:  3
actual sentiment manual label: 2
1
total_deviation 5
sentiment gauge guess:  3
actual sentiment manual label: 4
-1
total_deviation 4
sentiment gauge guess:  6
actual sentiment manual label: 7
-1
total_deviation 3
sentiment gauge guess:  4
actual sentiment manual label: 3
1
total_deviation 4
sentiment gauge guess:  6
actual sentiment manual label: 4
2
total_deviation 6
sentiment gauge guess:  6
actual sentiment man

-3
total_deviation 25
sentiment gauge guess:  6
actual sentiment manual label: 8
-2
total_deviation 23
sentiment gauge guess:  3
actual sentiment manual label: 3
0
total_deviation 23
sentiment gauge guess:  4
actual sentiment manual label: 4
0
total_deviation 23
sentiment gauge guess:  6
actual sentiment manual label: 9
-3
total_deviation 20
sentiment gauge guess:  3
actual sentiment manual label: 4
-1
total_deviation 19
sentiment gauge guess:  6
actual sentiment manual label: 3
3
total_deviation 22
sentiment gauge guess:  3
actual sentiment manual label: 8
-5
total_deviation 17
sentiment gauge guess:  3
actual sentiment manual label: 9
-6
total_deviation 11
sentiment gauge guess:  3
actual sentiment manual label: 8
-5
total_deviation 6
sentiment gauge guess:  3
actual sentiment manual label: 4
-1
total_deviation 5
sentiment gauge guess:  3
actual sentiment manual label: 4
-1
total_deviation 4
sentiment gauge guess:  6
actual sentiment manual label: 4
2
total_deviation 6
sentiment gaug